In [1]:
# --running
import os 
import sys 
os.environ["PYSPARK_PYTHON"] = "/opt/cloudera/parcels/Anaconda/bin/python" 
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_232-cloudera/jre" 
os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2- 1.cdh5.13.3.p0.316101/lib/spark2/" 
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib" 
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip") 
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip") 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 
from pyspark.sql.types import * 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1674390853434_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# This code loads data from kafka queue and insert records in mysql 
# table
# we are using pykafka and mysql connect to connect kafka and mysql

import sys
import mysql.connector
from pykafka import KafkaClient
from pykafka.common import OffsetType
from pykafka.exceptions import SocketDisconnectedError, LeaderNotAvailable
class KafkaMySQLSink:
    def __init__(self, kafka_bootstrap_server, kafka_topic_name, database_host,database_username, database_password,database_name):
    # Initialize Kafka Consumer
        kafka_client = KafkaClient(kafka_bootstrap_server)
        self.consumer = kafka_client.topics[kafka_topic_name].get_simple_consumer(consumer_group="groupid",auto_offset_reset=OffsetType.LATEST)
        print('#########################################################')
        # Initialize MySQL database connection
        self.db = mysql.connector.connect(host=database_host,user=database_username,password=database_password,database=database_name)
        db_cursor = self.db.cursor()
    def printing(self):
        print('--executed this line--')
    # Process single row
    def process_row(self, msg):
        # Get the db cursor
        db_cursor = self.db.cursor()
        # DB query for supporting UPSERT operation
        text = msg['text']
        category = msg['category']
        keywords = msg['keywords']
        campaign_id = msg['campaign_id']
        action = msg['action']
        target_gender = msg['target_gender']
        target_age_start = msg['target_age_range']['start']
        target_age_end = msg['target_age_range']['end']
        target_city = msg['target_city']
        target_state = msg['target_state']
        target_country = msg['target_country']
        target_income_bucket = msg['target_income_bucket']
        target_device = msg['target_device']
        cpc = msg['cpc']
        cpa = msg['cpa']
        budget = msg['budget']
        date_range_start = msg['date_range']['start']
        date_range_end = msg['date_range']['end']
        time_range_start  = msg['time_range']['start']
        time_range_end = msg['time_range']['end']
        # DB query for supporting UPSERT operation


        sql =  f"CALL ads_procedure ( \
        '{text}', \
        '{category}', \
        '{keywords}', \
        '{campaign_id}', \
        '{action}', \
        '{target_gender}', \
        {target_age_start}, \
        {target_age_end}, \
        '{target_city}', \
        '{target_state}', \
        '{target_country}', \
        '{target_income_bucket}', \
        '{target_device}', \
        {cpc}, \
        {cpa}, \
        {budget}, \
        '{date_range_start}', \
        '{date_range_end}', \
        '{time_range_start}', \
        '{time_range_end}');"
        #val = (text, text)
        self.db_cursor.execute(sql)
        # Commit the operation, so that it reflects globally
        self.db.commit()
        
    # Process kafka queue messages
    def process_events(self):
        try:
            for queue_message in self.consumer:
                if queue_message is not None:
                    msg = queue_message.value
                    print(type(msg))
                    print(msg)
                    #self.process_row(msg)
                    # In case Kafka connection errors, restart consumer ans start processing
        except (SocketDisconnectedError, LeaderNotAvailable) as e:
            self.consumer.stop()
            self.consumer.start()
            self.process_events()
        def __del__(self):
        # Cleanup consumer and database connection before termination
            self.consumer.stop()
            self.db_cursor.close()
            self.db.close()
   
if __name__ == "__main__":
# Validate Command line arguments
    if len(sys.argv) != 7:
        print('all_arguments given')
        print("Usage: kafka_mysql.py <kafka_bootstrap_server> <kafka_topic> <database_host> "  "<database_username> <database_password> <database_name")
        exit(-1)
        kafka_bootstrap_server = sys.argv[1]
        kafka_topic = sys.argv[2]
        database_host = sys.argv[3]
        database_username = sys.argv[4]
        database_password = sys.argv[5]
        database_name = sys.argv[6]
        ad_manager = None
        """
        kafka_bootstrap_server = '18.211.252.152:9092'
        kafka_topic = 'de-capstone1'
        database_host = 'localhost'
        database_username = 'root'
        database_password = 'root'
        database_name = 'capstone'
        ad_manager = None
        """
        try:
           
            kafka_mysql_sink = KafkaMySQLSink('18.211.252.152:9092','de-capstone1','ec2-54-90-242-117.compute-1.amazonaws.com','root','123', 'capstone')
            kafka_mysql_sink.printing()
            kafka_mysql_sink.process_events()
        except KeyboardInterrupt:
            print('KeyboardInterrupt, exiting...')
        finally:
            if kafka_mysql_sink is not None:
                del kafka_mysql_sink
          



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
kafka_mysql_sink = KafkaMySQLSink('18.211.252.152:9092','de-capstone1','ec2-54-90-242-117.compute-1.amazonaws.com','root','123', 'capstone')
kafka_mysql_sink.printing()
kafka_mysql_sink.process_events()
if kafka_mysql_sink is not None:
    del kafka_mysql_sink

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…